# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-af7b0d4e63-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Horror Franchises explorative search") 

Consider the following exploratory scenario:


> we are interested in horror movies, in particular we care about the slasher subgenre, and we want to compare the Halloween and the Friday the 13th francise. 

Background knowledge useful for better conducting the search: Halloween is a movie from 1978 with many sequels and the set of Halloween + all the sequels is called the "Halloween franchise". The same considerations hold for Friday the 13th.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q221103`     | Halloween       | node |
| `wd:Q1243029`| Friday the 13th       | node |





Also consider

```
wd:Q1243029 ?p ?obj .
```

is the BGP to retrieve all **properties of Friday the 13th**

The workload should


1. Investigate the workers of the two original movies and check any commonality 

2. Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)

3. Check which actor/actress did more movies after the release of these two films (consider the films separately)

4. Get all the movies of the two franchises (return year and director)

5. Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)? 

6. Investigate which franchise is highly regarded by the critics overall

7. Investigate which franchise costed more 

In [1]:
# start your workflow here

In [2]:
queryString = """
SELECT *
WHERE { 

wd:Q1243029 ?p ?obj .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.gov/authorities/names/no98033420')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3212'), ('obj', 'urn:isan:0000-0000-31EA-0000-G-0000-0000-Q')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P4276'), ('obj', 'https://data.cinematheque.qc.ca/data/Work1467')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/016mng')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('obj', 'friday13th')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('obj', 'm/friday_the_13th_part_1')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('obj', '505')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('obj', 'http://www.wikidata.org/entity/Q1067324')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('obj', 'http://www.wikidata.org/ent

118

<h3> TASK 1 STARTS HERE </h3>
Investigate the workers of the two original movies and check any commonality.

Retrieve some properties of Friday 13th.

In [5]:
#Q1
#get labels
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q1243029 ?p ?obj .

#Labels
?p <http://schema.org/name> ?pname .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pname', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P161'), ('pname', 'cast member')]
[('p', 'http://www.wikidata.org/prop/direct/P162'), ('pname', 'producer')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pname', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P175'), ('pname', 'performer')]
[('p', 'http://w

93

In the resultset above I found the following interesting properties: P161 cast member, P162 producer, P175 performer, P272 production company, P344 director of photography
P57 director, P58 screenwriter, P86 composer, P2554 production designer. Retrieve the cast of Fridat 13th.

In [6]:
#Q2
#useful predicates found above: P161 cast member, P162 producer, P175 performer, P272 production company, P344 director of photography
#P57 director, P58 screenwriter, P86 composer, P2554 production designer
#retrieve Friday 13th cast
queryString = """
SELECT *
WHERE { 

wd:Q1243029 wdt:P161 ?cast.
} 
"""

print("Results")
run_query(queryString)

Results
[('cast', 'http://www.wikidata.org/entity/Q2440012')]
[('cast', 'http://www.wikidata.org/entity/Q263765')]
[('cast', 'http://www.wikidata.org/entity/Q3428829')]
[('cast', 'http://www.wikidata.org/entity/Q3434272')]
[('cast', 'http://www.wikidata.org/entity/Q3454165')]
[('cast', 'http://www.wikidata.org/entity/Q13218111')]
[('cast', 'http://www.wikidata.org/entity/Q2570201')]
[('cast', 'http://www.wikidata.org/entity/Q3047636')]
[('cast', 'http://www.wikidata.org/entity/Q524486')]


9

Retrieve other workers such as producer, composer, director.

In [7]:
#Q3
#other workers of friday 13th
queryString = """
SELECT *
WHERE { 

wd:Q1243029 wdt:P162 ?producer;
            wdt:P175 ?performer;
            wdt:P344 ?pdir;
            wdt:P57 ?dir;
            wdt:P58 ?swr;
            wdt:P86 ?composer;
            wdt:P2554 ?prod.
} 
"""

print("Results")
run_query(queryString)

Results
[('producer', 'http://www.wikidata.org/entity/Q709325'), ('performer', 'http://www.wikidata.org/entity/Q664025'), ('pdir', 'http://www.wikidata.org/entity/Q4863956'), ('dir', 'http://www.wikidata.org/entity/Q709325'), ('swr', 'http://www.wikidata.org/entity/Q7926158'), ('composer', 'http://www.wikidata.org/entity/Q664025'), ('prod', 'http://www.wikidata.org/entity/Q450964')]


1

Check if some actors acted in both films.

In [8]:
#Q4
#check if some actor acted in both films
queryString = """
SELECT *
WHERE { 

wd:Q1243029 wdt:P161 ?cast.
wd:Q221103 wdt:P161 ?cast.
} 
"""

print("Results")
run_query(queryString)

Results
Empty


0

Retrieve some predicates of the director of Friday 13th.

In [10]:
#Q5
#check some predicates related to director
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q1243029 wdt:P57 ?dir.
?dir ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('pname', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('pname', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('pname', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('pname', 'place of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P1969'), ('pname', 'MovieMeter director ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P2019'), ('pname', 'AllMovie person ID')]
[('p', 'http://www.wikidata.org/p

50

Retrieve for each country, how many workers of Friday 13th are from there.

In [11]:
#Q6
#country distrubution for friday 13th workers
queryString = """
SELECT ?country (COUNT(DISTINCT ?o) AS ?howMany)
{  wd:Q1243029 ?p ?o.
   ?o wdt:P27 ?country.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?country)
"""
print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('howMany', '13')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('howMany', '1')]


2

Same but for Halloween.

In [12]:
#Q7
#country distrubution for hallowen workers
queryString = """
SELECT ?country (COUNT(DISTINCT ?o) AS ?howMany)
{  wd:Q221103 ?p ?o.
   ?o wdt:P27 ?country.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?country)
"""
print("Results")
run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q30'), ('howMany', '16')]
[('country', 'http://www.wikidata.org/entity/Q858'), ('howMany', '1')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('howMany', '1')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('howMany', '1')]


4

Both casts almost are made of american actors. Retrieve how many workers have been nominated for an award for each film.

In [39]:
#cast sizes are similar, also almost every worker is from Q30
#Q8
#how many workers for each film were nomitated for some awards(using P1411 found in Q5)
queryString = """
SELECT ?film (COUNT(DISTINCT ?o) AS ?howMany)
{  ?film ?p ?o.
   ?o wdt:P1411 ?award.
   VALUES ?film {wd:Q221103 wd:Q1243029}
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?film)
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('howMany', '3')]
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('howMany', '2')]


2

Compute how many men and women were involved in the production of Halloween.

In [14]:
#Q9
#count gender
queryString = """
SELECT ?gender (COUNT(?o) AS ?howMany)
{  wd:Q221103 ?p ?o.
   ?o wdt:P21 ?gender.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?gender)
"""
print("Results")
run_query(queryString)

Results
[('gender', 'http://www.wikidata.org/entity/Q6581097'), ('howMany', '16')]
[('gender', 'http://www.wikidata.org/entity/Q6581072'), ('howMany', '7')]


2

Same but for Friday 13th.

In [15]:
#Q10
#count gender
queryString = """
SELECT ?gender (COUNT(?o) AS ?howMany)
{  wd:Q1243029 ?p ?o.
   ?o wdt:P21 ?gender.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?gender)
"""
print("Results")
run_query(queryString)

Results
[('gender', 'http://www.wikidata.org/entity/Q6581097'), ('howMany', '10')]
[('gender', 'http://www.wikidata.org/entity/Q6581072'), ('howMany', '6')]


2

<h3> TASK 1 ENDS HERE </h3>
I did a comparison between the two casts and i found out that they have similar sizes, both are almost made of american workers.
Also in both cases there are more men than women. On top of that the Halloween workers have on more actor who was nomitated to some awards.

<h3> TASK 2 STARTS HERE </h3>
Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release. 

Retrieve some actors' predicates.

In [19]:
#Q11
#look for predicates related to acting/partecipating in films
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q1243029 wdt:P161 ?cast.
?cast ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P1196'), ('pname', 'manner of death')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('pname', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1220'), ('pname', 'Internet Broadway Database person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('pname', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1266'), ('pname', 'AlloCiné person ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('pname', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1283'), ('pname', 'filmography')]
[('p', 'http://www.wikidata.org/prop/direct/P1284'), ('pname'

100

Trying the reverse path (from film to actor) to count in how many films the actors of friday 13th acted before the release.

In [20]:
#Q12
#maybe the reverse path is better (from film to actor)
#count in how many films the actors of friday 13th acted before the release
queryString = """
SELECT ?actor (COUNT(DISTINCT ?film) AS ?films)
WHERE { 

wd:Q1243029 wdt:P161 ?actor;
            wdt:P577 ?datef.
?film wdt:P161 ?actor;
      wdt:P577 ?date.

FILTER(?date<?datef)
} 
GROUP BY (?actor)
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q263765'), ('films', '7')]
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('films', '4')]
[('actor', 'http://www.wikidata.org/entity/Q3428829'), ('films', '4')]
[('actor', 'http://www.wikidata.org/entity/Q2440012'), ('films', '3')]
[('actor', 'http://www.wikidata.org/entity/Q3434272'), ('films', '2')]
[('actor', 'http://www.wikidata.org/entity/Q13218111'), ('films', '1')]
[('actor', 'http://www.wikidata.org/entity/Q2570201'), ('films', '1')]
[('actor', 'http://www.wikidata.org/entity/Q524486'), ('films', '1')]
[('actor', 'http://www.wikidata.org/entity/Q3047636'), ('films', '1')]


9

Use P166 (award received) found in Q11.

In [21]:
#Q13
#check P166 found in Q11
queryString = """
SELECT ?actor ?award
WHERE { 

wd:Q1243029 wdt:P161 ?actor.
?actor wdt:P166 ?award.

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q251542')]
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q17985761')]
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q105447')]
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q18600734')]


4

Retrieve award properties to see if there is a temporal reference.

In [22]:
#Q14
#looking for a temporal reference related to each award
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

wd:Q1243029 wdt:P161 ?actor.
?actor wdt:P166 ?award.
?award ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('pname', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('pname', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1705'), ('pname', 'native label')]
[('p', 'http://www.wikidata.org/prop/direct/P179'), ('pname', 'part of the series')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pname', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P180'), ('pname', 'depicts')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('pname', 'has list')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pname', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname

29

We have P580(start time) and P585(point in time) that could be useful. Check what results we get using them.

In [23]:
#Q15
#p580 and p585 seem useful
queryString = """
SELECT DISTINCT ?time ?point
WHERE { 

wd:Q1243029 wdt:P161 ?actor.
?actor wdt:P166 ?award.
?award wdt:P580 ?time;
       wdt:P585 ?point.


} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Retrieve start time of an award won by an actor of Friday 13th.

In [25]:
#Q16
#p580 and p585 seem useful
queryString = """
SELECT DISTINCT ?time ?point
WHERE { 

wd:Q1243029 wdt:P161 ?actor.
?actor wdt:P166 ?award.
?award wdt:P580 ?time.
OPTIONAL {?award wdt:P585 ?point}


} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('time', '1972-01-01T00:00:00Z')]


1

As above but retrieve the point in time.

In [26]:
#Q17
#reverse of previous one
queryString = """
SELECT DISTINCT ?time ?point
WHERE { 

wd:Q1243029 wdt:P161 ?actor.
?actor wdt:P166 ?award.
?award wdt:P585 ?time.
OPTIONAL {?award wdt:P580 ?point}


} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('time', '2015-01-25T00:00:00Z')]


1

Retrieve the actors of Friday 13th who won an award and the time when he did it.

In [41]:
#Q18
queryString = """
SELECT ?actor ?award  ?aname ?time ?point
WHERE { 

wd:Q1243029 wdt:P161 ?actor.
?actor wdt:P166 ?award.
OPTIONAL {?award wdt:P580 ?time}
OPTIONAL {?award wdt:P585 ?point}

#Labels
?award <http://schema.org/name> ?aname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q105447'), ('aname', 'Saturn Award'), ('time', '1972-01-01T00:00:00Z')]
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q18600734'), ('aname', '21st Screen Actors Guild Awards'), ('point', '2015-01-25T00:00:00Z')]
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q251542'), ('aname', 'Golden Globe Award for Best Actor – Miniseries or Television Film')]
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('award', 'http://www.wikidata.org/entity/Q17985761'), ('aname', 'star on Hollywood Walk of Fame')]


4

Retrieve actor who won an award before the release of Friday 13th.

In [32]:
#Q19
#retrieve actor who won an award before the release of friday 13th
queryString = """
SELECT ?actor (COUNT(DISTINCT (?award)) AS ?awards)
WHERE { 

wd:Q1243029 wdt:P161 ?actor;
            wdt:P577 ?date.
?actor wdt:P166 ?award.
?award wdt:P580 ?time.

FILTER(?time<?date)
} 
GROUP BY (?actor)
ORDER BY (?awards)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('awards', '1')]


1

Count in how many films the actors of Halloween acted before the release.

In [33]:
#Q20
#as Q12 but for Halloween
queryString = """
SELECT ?actor (COUNT(DISTINCT ?film) AS ?films)
WHERE { 

wd:Q221103 wdt:P161 ?actor;
            wdt:P577 ?datef.
?film wdt:P161 ?actor;
      wdt:P577 ?date.

FILTER(?date<?datef)
} 
GROUP BY (?actor)
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q314914'), ('films', '80')]
[('actor', 'http://www.wikidata.org/entity/Q3093664'), ('films', '25')]
[('actor', 'http://www.wikidata.org/entity/Q2865227'), ('films', '12')]
[('actor', 'http://www.wikidata.org/entity/Q1064022'), ('films', '7')]
[('actor', 'http://www.wikidata.org/entity/Q270079'), ('films', '6')]
[('actor', 'http://www.wikidata.org/entity/Q1853959'), ('films', '5')]
[('actor', 'http://www.wikidata.org/entity/Q95008'), ('films', '3')]
[('actor', 'http://www.wikidata.org/entity/Q201570'), ('films', '2')]
[('actor', 'http://www.wikidata.org/entity/Q6248629'), ('films', '2')]
[('actor', 'http://www.wikidata.org/entity/Q936000'), ('films', '2')]
[('actor', 'http://www.wikidata.org/entity/Q106997'), ('films', '1')]
[('actor', 'http://www.wikidata.org/entity/Q6963012'), ('films', '1')]
[('actor', 'http://www.wikidata.org/entity/Q3066766'), ('films', '1')]


13

Retrieve actor who won an award before the release of Halloween.

In [34]:
#Q21
#as Q19 but for halloween
queryString = """
SELECT ?actor (COUNT(DISTINCT (?award)) AS ?awards)
WHERE { 

wd:Q221103 wdt:P161 ?actor;
            wdt:P577 ?date.
?actor wdt:P166 ?award.
?award wdt:P580 ?time.

FILTER(?time<?date)
} 
GROUP BY (?actor)
ORDER BY (?awards)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
Empty


0

Same above but using Optional for time property.

In [37]:
#Q22
#put time as optional
queryString = """
SELECT DISTINCT ?actor ?award ?aname ?time
WHERE { 

wd:Q221103 wdt:P161 ?actor;
            wdt:P577 ?date.
?actor wdt:P166 ?award.
OPTIONAL {?award wdt:P580 ?time.}

#Labels
?award <http://schema.org/name> ?aname .

} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q314914'), ('award', 'http://www.wikidata.org/entity/Q10762848'), ('aname', 'Officer of the Order of the British Empire')]
[('actor', 'http://www.wikidata.org/entity/Q106997'), ('award', 'http://www.wikidata.org/entity/Q787123'), ('aname', 'BAFTA Award for Best Actress in a Supporting Role')]
[('actor', 'http://www.wikidata.org/entity/Q106997'), ('award', 'http://www.wikidata.org/entity/Q1011564'), ('aname', 'Golden Globe Award for Best Actress – Motion Picture Musical or Comedy')]
[('actor', 'http://www.wikidata.org/entity/Q106997'), ('award', 'http://www.wikidata.org/entity/Q17985761'), ('aname', 'star on Hollywood Walk of Fame')]


4

<h3> TASK 2 ENDS HERE </h3>
Donald Pleasence is the actor who acted more times before the release of Halloween. On the other hand the actor who acted in more films before the release
of Friday 13th is Kevin Bacon. To conclude, Betsy Palmer is the only actor who managed to get an award before acting in Friday 13th.

In [40]:
#TASK 2 ENDS HERE:
#Q314914 more film before acting in hallowen
#Q3454165 only one to get an awards before acting in friday 13th
#Q263765 acted in more films than anyone else before the releasing of friday 13th
#i have no temporal references to retrieve the actor who won more awards before acting in halloween

#gets names
queryString = """
SELECT DISTINCT ?s ?sname
WHERE { 

?s ?p ?o.
VALUES ?s {wd:Q314914  wd:Q3454165 wd:Q263765}

#Labels
?s <http://schema.org/name> ?sname .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q263765'), ('sname', 'Betsy Palmer')]
[('s', 'http://www.wikidata.org/entity/Q314914'), ('sname', 'Donald Pleasence')]
[('s', 'http://www.wikidata.org/entity/Q3454165'), ('sname', 'Kevin Bacon')]


3

<h3> TASK 3 STARTS HERE </h3>
Check which actor/actress did more movies after the release of these two films (consider the films separately).

Retrieve who acted in more films after the release of Friday 13th.

In [45]:
#Q23
#who acted in more movies after the release of friday 13th
queryString = """
SELECT ?actor (COUNT(DISTINCT ?film) AS ?films)
WHERE { 

?film wdt:P161 ?actor;
      wdt:P577 ?date.
wd:Q1243029 wdt:P577 ?datef.

{SELECT ?actor WHERE{  
?f wdt:P161 ?actor.
VALUES ?f {wd:Q221103 wd:Q1243029}
}}

FILTER(?date>?datef)
} 
GROUP BY (?actor)
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('films', '64')]
[('actor', 'http://www.wikidata.org/entity/Q314914'), ('films', '50')]
[('actor', 'http://www.wikidata.org/entity/Q106997'), ('films', '50')]
[('actor', 'http://www.wikidata.org/entity/Q95008'), ('films', '17')]
[('actor', 'http://www.wikidata.org/entity/Q270079'), ('films', '16')]
[('actor', 'http://www.wikidata.org/entity/Q2440012'), ('films', '9')]
[('actor', 'http://www.wikidata.org/entity/Q2865227'), ('films', '8')]
[('actor', 'http://www.wikidata.org/entity/Q263765'), ('films', '8')]
[('actor', 'http://www.wikidata.org/entity/Q3093664'), ('films', '7')]
[('actor', 'http://www.wikidata.org/entity/Q3434272'), ('films', '6')]
[('actor', 'http://www.wikidata.org/entity/Q1064022'), ('films', '6')]
[('actor', 'http://www.wikidata.org/entity/Q1853959'), ('films', '5')]
[('actor', 'http://www.wikidata.org/entity/Q936000'), ('films', '5')]
[('actor', 'http://www.wikidata.org/entity/Q3428829'), ('films', '4')]


21

Retrieve who acted in more movies after the release of Halloween.

In [46]:
#Q24
#who acted in more movies after the release of halloween
queryString = """
SELECT ?actor (COUNT(DISTINCT ?film) AS ?films)
WHERE { 

?film wdt:P161 ?actor;
      wdt:P577 ?date.
wd:Q221103 wdt:P577 ?datef.

{SELECT ?actor WHERE{  
?f wdt:P161 ?actor.
VALUES ?f {wd:Q221103 wd:Q1243029}
}}

FILTER(?date>?datef)
} 
GROUP BY (?actor)
ORDER BY DESC(?films)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('actor', 'http://www.wikidata.org/entity/Q3454165'), ('films', '67')]
[('actor', 'http://www.wikidata.org/entity/Q314914'), ('films', '55')]
[('actor', 'http://www.wikidata.org/entity/Q106997'), ('films', '52')]
[('actor', 'http://www.wikidata.org/entity/Q270079'), ('films', '20')]
[('actor', 'http://www.wikidata.org/entity/Q95008'), ('films', '18')]
[('actor', 'http://www.wikidata.org/entity/Q3093664'), ('films', '12')]
[('actor', 'http://www.wikidata.org/entity/Q2865227'), ('films', '10')]
[('actor', 'http://www.wikidata.org/entity/Q2440012'), ('films', '10')]
[('actor', 'http://www.wikidata.org/entity/Q1064022'), ('films', '9')]
[('actor', 'http://www.wikidata.org/entity/Q1853959'), ('films', '8')]
[('actor', 'http://www.wikidata.org/entity/Q263765'), ('films', '8')]
[('actor', 'http://www.wikidata.org/entity/Q3434272'), ('films', '6')]
[('actor', 'http://www.wikidata.org/entity/Q936000'), ('films', '6')]
[('actor', 'http://www.wikidata.org/entity/Q3428829'), ('films', '6'

22

<h3> TASK 3 ENDS HERE </h3>
The two actors with the most partecipations are the same as TASK 2. 
It's interesting to notice that their number of appearances is more then the triple of their colleagues.
Also Betty Palmer who was the one only to get an award before Friday 13th release is not even in the top 10.

Count in how many films Donald Pleasance acted in.

In [3]:
#Q25
#just for curiosity, count the film with Donald Pleasance
queryString = """
SELECT (COUNT (DISTINCT (?film)) AS ?films) WHERE
{
?film wdt:P161 wd:Q314914
}
"""
print("Results")
run_query(queryString)

Results
[('films', '128')]


1

<h3>TASK 4 STARTS HERE</h3>
Get all the movies of the two franchises (return year and director)

In Q1 i found P179 (part of series), let's see if it is useful.

In [5]:
#Q26
#in Q1 i found P179 (part of series), let's see if it is useful
queryString = """
SELECT ?o ?oname WHERE{

wd:Q1243029 wdt:P179 ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 

LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1138878'), ('oname', 'Friday the 13th')]


1

Using P179+Q1138878 i should find all the films belonging to Friday 13th saga. Let's check.

In [6]:
#Q27
#so using P179+Q1138878 i should find all the films belonging to Friday 13th saga
queryString = """
SELECT ?film ?sname ?director WHERE{

?film wdt:P179 wd:Q1138878;
        wdt:P57  ?director.

#Labels
?film <http://schema.org/name> ?sname .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('sname', 'Friday the 13th Part 2'), ('director', 'http://www.wikidata.org/entity/Q709325')]
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('sname', 'Friday the 13th Part 2'), ('director', 'http://www.wikidata.org/entity/Q107404')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('sname', 'Friday the 13th Part III'), ('director', 'http://www.wikidata.org/entity/Q107404')]
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('sname', 'Friday the 13th'), ('director', 'http://www.wikidata.org/entity/Q709325')]
[('film', 'http://www.wikidata.org/entity/Q1427521'), ('sname', 'Jason X'), ('director', 'http://www.wikidata.org/entity/Q472868')]
[('film', 'http://www.wikidata.org/entity/Q1683918'), ('sname', 'Jason Goes to Hell: The Final Friday'), ('director', 'http://www.wikidata.org/entity/Q4679450')]
[('film', 'http://www.wikidata.org/entity/Q1454794'), ('sname', 'Friday the 13th Part VIII: Jason Takes Manhattan')

13

Use group_concat since a film can have more than one director, same for date of pubblication.

In [7]:
#Q28
#use group_concat since a film can have more than one director, same for date of pubblication
queryString = """
SELECT ?film ?fname ?directors ?year WHERE{
{
SELECT ?film (GROUP_CONCAT(?director;SEPARATOR=",") AS ?directors) WHERE{

?film wdt:P179 wd:Q1138878;
        wdt:P57  ?director.
}
GROUP BY (?film)}
{SELECT ?film (GROUP_CONCAT(?date;SEPARATOR=",") AS ?year) WHERE{

?film wdt:P179 wd:Q1138878;
        wdt:P577 ?date.
}
GROUP BY (?film)}
#Labels
?film <http://schema.org/name> ?fname .
}

ORDER BY (?year)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('fname', 'Friday the 13th'), ('directors', 'http://www.wikidata.org/entity/Q709325'), ('year', '1980-05-09 00:00:00Z,1980-10-23 00:00:00Z,1980-11-10 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('fname', 'Friday the 13th Part 2'), ('directors', 'http://www.wikidata.org/entity/Q709325,http://www.wikidata.org/entity/Q107404'), ('year', '1981-04-30 00:00:00Z,1981-11-02 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('fname', 'Friday the 13th Part III'), ('directors', 'http://www.wikidata.org/entity/Q107404'), ('year', '1982-08-13 00:00:00Z,1983-05-12 00:00:00Z,1983-05-13 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q1243019'), ('fname', 'Friday the 13th: The Final Chapter'), ('directors', 'http://www.wikidata.org/entity/Q11335'), ('year', '1984-01-01 00:00:00Z,1984-07-13 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q286830'), ('fname', 'Friday the 13th: A New Beginn

12

Find Halloween saga.

In [8]:
#Q29
#find Halloween saga
queryString = """
SELECT ?o ?oname WHERE{

wd:Q221103 wdt:P179 ?o.

#Labels
?o <http://schema.org/name> ?oname .

} 

LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q1364022'), ('oname', 'Halloween')]


1

Retrieve directors and date of pubblication for all films belonging to Halloween saga.

In [10]:
#Q30
#as above but for Halloween saga
queryString = """
SELECT ?film ?fname ?directors ?year WHERE{
{
SELECT ?film (GROUP_CONCAT(?director;SEPARATOR=",") AS ?directors) WHERE{

?film wdt:P179 wd:Q1364022;
        wdt:P57  ?director.
}
GROUP BY (?film)}
{SELECT ?film (GROUP_CONCAT(?date;SEPARATOR=",") AS ?year) WHERE{

?film wdt:P179 wd:Q1364022;
        wdt:P577 ?date.
}
GROUP BY (?film)}
#Labels
?film <http://schema.org/name> ?fname .
}

ORDER BY (?year)
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('fname', 'Halloween'), ('directors', 'http://www.wikidata.org/entity/Q95008'), ('year', '1978-10-25 00:00:00Z,1979-07-06 00:00:00Z,1979-08-06 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q850312'), ('fname', 'Halloween II'), ('directors', 'http://www.wikidata.org/entity/Q95008,http://www.wikidata.org/entity/Q1384404'), ('year', '1981-01-01 00:00:00Z,1982-11-11 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('fname', 'Halloween III: Season of the Witch'), ('directors', 'http://www.wikidata.org/entity/Q574983'), ('year', '1982-01-01 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q959853'), ('fname', 'Halloween 4: The Return of Michael Myers'), ('directors', 'http://www.wikidata.org/entity/Q1268483'), ('year', '1988-01-01 00:00:00Z,1989-05-18 00:00:00Z')]
[('film', 'http://www.wikidata.org/entity/Q976728'), ('fname', 'Halloween 5: The Revenge of Michael Myers'), ('directors', 'http://www.wik

10

<h3> TASK 4 ENDS HERE </h3>

<h3> TASK 5 STARTS HERE</h3>
Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)?

Retrieve the most involved in Friday 13th franchise.

In [2]:
#Q31
#most involved in Friday 13th franchise (?film wdt:P179 wd:Q1138878)
queryString = """
SELECT ?o (COUNT(DISTINCT ?film) AS ?Appearances)
{  
?film wdt:P179 wd:Q1138878. #franchise
?film ?p ?o.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?o)
ORDER BY DESC(?Appearances)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q664025'), ('Appearances', '9')]
[('o', 'http://www.wikidata.org/entity/Q709325'), ('Appearances', '6')]
[('o', 'http://www.wikidata.org/entity/Q465134'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q3082711'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q375419'), ('Appearances', '3')]
[('o', 'http://www.wikidata.org/entity/Q263765'), ('Appearances', '3')]
[('o', 'http://www.wikidata.org/entity/Q30327831'), ('Appearances', '2')]
[('o', 'http://www.wikidata.org/entity/Q3063036'), ('Appearances', '2')]
[('o', 'http://www.wikidata.org/entity/Q3086901'), ('Appearances', '2')]
[('o', 'http://www.wikidata.org/entity/Q6234410'), ('Appearances', '2')]


10

Retrieve in which films and with what roles appearead Harry Manfredini (first result of previous query).

In [4]:
#Q32
#have a look at wd:Q664025
queryString = """
SELECT ?name ?pname ?film ?fname
{  
?film wdt:P179 wd:Q1138878. #franchise
?film ?p wd:Q664025.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
   
#Labels
wd:Q664025 <http://schema.org/name> ?name .
?p <http://schema.org/name> ?pname .
?film <http://schema.org/name> ?fname .
}

LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('name', 'Harry Manfredini'), ('pname', 'performer'), ('film', 'http://www.wikidata.org/entity/Q1454799'), ('fname', 'Friday the 13th Part VI: Jason Lives')]
[('name', 'Harry Manfredini'), ('pname', 'performer'), ('film', 'http://www.wikidata.org/entity/Q1243013'), ('fname', 'Friday the 13th Part III')]
[('name', 'Harry Manfredini'), ('pname', 'performer'), ('film', 'http://www.wikidata.org/entity/Q1243019'), ('fname', 'Friday the 13th: The Final Chapter')]
[('name', 'Harry Manfredini'), ('pname', 'performer'), ('film', 'http://www.wikidata.org/entity/Q1243009'), ('fname', 'Friday the 13th Part 2')]
[('name', 'Harry Manfredini'), ('pname', 'performer'), ('film', 'http://www.wikidata.org/entity/Q1243029'), ('fname', 'Friday the 13th')]
[('name', 'Harry Manfredini'), ('pname', 'performer'), ('film', 'http://www.wikidata.org/entity/Q286830'), ('fname', 'Friday the 13th: A New Beginning')]
[('name', 'Harry Manfredini'), ('pname', 'composer'), ('film', 'http://www.wikidata.org/enti

15

A worker can be involved in different roles, so it's better to not use DISTINCT. Retrieve the same query as Q31 but without distinct.

In [5]:
#Q33
#as Q31 but without DISTINCT since someone can be involved with different roles
queryString = """
SELECT ?o (COUNT(?film) AS ?Appearances)
{  
?film wdt:P179 wd:Q1138878. #franchise
?film ?p ?o.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?o)
ORDER BY DESC(?Appearances)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q664025'), ('Appearances', '15')]
[('o', 'http://www.wikidata.org/entity/Q709325'), ('Appearances', '7')]
[('o', 'http://www.wikidata.org/entity/Q107404'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q3082711'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q465134'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q375419'), ('Appearances', '3')]
[('o', 'http://www.wikidata.org/entity/Q263765'), ('Appearances', '3')]
[('o', 'http://www.wikidata.org/entity/Q3116155'), ('Appearances', '2')]
[('o', 'http://www.wikidata.org/entity/Q1314250'), ('Appearances', '2')]
[('o', 'http://www.wikidata.org/entity/Q30327831'), ('Appearances', '2')]


10

Retrieve the most involved in most films of Halloween franchise.

In [7]:
#Q34
#most involved in Halloween franchise
queryString = """
SELECT ?o (COUNT(DISTINCT ?film) AS ?Appearances)
{  
?film wdt:P179 wd:Q1364022. #franchise
?film ?p ?o.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?o)
ORDER BY DESC(?Appearances)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q314914'), ('Appearances', '5')]
[('o', 'http://www.wikidata.org/entity/Q106997'), ('Appearances', '5')]
[('o', 'http://www.wikidata.org/entity/Q95008'), ('Appearances', '5')]
[('o', 'http://www.wikidata.org/entity/Q380471'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q262102'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q272944'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q730362'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q1378684'), ('Appearances', '3')]
[('o', 'http://www.wikidata.org/entity/Q201570'), ('Appearances', '3')]
[('o', 'http://www.wikidata.org/entity/Q6963012'), ('Appearances', '3')]


10

Retrieve the worker with the highest number of roles in the Halloween franchise.

In [8]:
#Q35
#most involved in Halloween franchise without distinct
queryString = """
SELECT ?o (COUNT(?film) AS ?Appearances)
{  
?film wdt:P179 wd:Q1364022. #franchise
?film ?p ?o.
   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
}
GROUP BY (?o)
ORDER BY DESC(?Appearances)
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q95008'), ('Appearances', '13')]
[('o', 'http://www.wikidata.org/entity/Q262102'), ('Appearances', '6')]
[('o', 'http://www.wikidata.org/entity/Q298255'), ('Appearances', '6')]
[('o', 'http://www.wikidata.org/entity/Q314914'), ('Appearances', '5')]
[('o', 'http://www.wikidata.org/entity/Q106997'), ('Appearances', '5')]
[('o', 'http://www.wikidata.org/entity/Q730362'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q272944'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q380471'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q574983'), ('Appearances', '4')]
[('o', 'http://www.wikidata.org/entity/Q497036'), ('Appearances', '3')]


10

Retrieve in which films and with what role John Carpenter appeared in Halloween saga.

In [12]:
#Q36
#have a look at the most involved wd:Q95008 
queryString = """
SELECT ?name ?pname ?film ?fname
{  
?film wdt:P179 wd:Q1364022. #franchise
?film ?p wd:Q95008 .

   VALUES ?p {wdt:P161 wdt:P162 wdt:P175 wdt:P344 wdt:P57 wdt:P58 wdt:P86 wdt:P2554}
   
#Labels
wd:Q95008  <http://schema.org/name> ?name .
?p <http://schema.org/name> ?pname .
?film <http://schema.org/name> ?fname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('name', 'John Carpenter'), ('pname', 'cast member'), ('film', 'http://www.wikidata.org/entity/Q221103'), ('fname', 'Halloween')]
[('name', 'John Carpenter'), ('pname', 'producer'), ('film', 'http://www.wikidata.org/entity/Q850312'), ('fname', 'Halloween II')]
[('name', 'John Carpenter'), ('pname', 'producer'), ('film', 'http://www.wikidata.org/entity/Q203705'), ('fname', 'Halloween III: Season of the Witch')]
[('name', 'John Carpenter'), ('pname', 'performer'), ('film', 'http://www.wikidata.org/entity/Q221103'), ('fname', 'Halloween')]
[('name', 'John Carpenter'), ('pname', 'director'), ('film', 'http://www.wikidata.org/entity/Q850312'), ('fname', 'Halloween II')]
[('name', 'John Carpenter'), ('pname', 'director'), ('film', 'http://www.wikidata.org/entity/Q221103'), ('fname', 'Halloween')]
[('name', 'John Carpenter'), ('pname', 'screenwriter'), ('film', 'http://www.wikidata.org/entity/Q976728'), ('fname', 'Halloween 5: The Revenge of Michael Myers')]
[('name', 'John Carpenter

13

<h3> TASK 5 ENDS HERE </h3>
The most involved in Friday 13th franchise is Henry Manfredini, he is a perfomer and composer who took part in 9 films.
Meanwhile for Halloween franchise i found out that John Carpenter took part in five different films with several roles (actor, producer, performer,
director,screenwriter). The most involved actors are Donald Pleasance (already found in task 2 and 3) and Jamie Lee Curtis who acted in five films.

<h3> TASK 6 STARTS HERE </h3>
Investigate which franchise is highly regarded by the critics overall

In Q1 i found out P5021 (assessment), let's check if it can be useful.

In [16]:
#Q37
#in Q1 i found out P5021 (assessment), let's check if it can be useful
queryString = """
SELECT ?film ?o ?oname
{  
?film wdt:P179 wd:Q1364022. #franchise
?film wdt:P5021 ?o.
   
#Labels
?o <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q909063'), ('o', 'http://www.wikidata.org/entity/Q4165246'), ('oname', 'Bechdel test')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('o', 'http://www.wikidata.org/entity/Q4165246'), ('oname', 'Bechdel test')]
[('film', 'http://www.wikidata.org/entity/Q1050815'), ('o', 'http://www.wikidata.org/entity/Q4165246'), ('oname', 'Bechdel test')]
[('film', 'http://www.wikidata.org/entity/Q850312'), ('o', 'http://www.wikidata.org/entity/Q4165246'), ('oname', 'Bechdel test')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('o', 'http://www.wikidata.org/entity/Q85783379'), ('oname', 'Mako Mori test')]


5

Retrieve some properties of these tests to understand what they are about.

In [17]:
#Q37
#in Q1 i found out P5021 (assessment), let's check if it can be useful
queryString = """
SELECT ?p ?pname ?obj ?oname
{  
?film wdt:P179 wd:Q1364022. #franchise
?film wdt:P5021 ?o.
?o ?p ?obj.
   
#Labels
?p <http://schema.org/name> ?pname .
?obj <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q7252'), ('oname', 'feminism')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q7252'), ('oname', 'feminism')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q7252'), ('oname', 'feminism')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q7252'), ('oname', 'feminism')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q7252'), ('oname', 'feminism')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http://www.wikidata.org/entity/Q2265123'), ('oname', 'Feminist film theory')]
[('p', 'http://www.wikidata.org/prop/direct/P1269'), ('pname', 'facet of'), ('obj', 'http

50

They don't seem to be related to critics. Let's look if there is some info from Metacritic (P1712 found in Q1).

In [18]:
#Q38
#ok it doesn't seem to be related to critics
#let's look if there is some info from Metacritic (P1712 found in Q1)
queryString = """
SELECT ?p ?pname ?obj ?oname
{  
?film wdt:P179 wd:Q1364022. #franchise
?film wdt:P1712 ?o.
?o ?p ?obj.
   
#Labels
?p <http://schema.org/name> ?pname .
?obj <http://schema.org/name> ?oname .
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
Empty


0

In [19]:
#Q39
#let's look if there is some info from Metacritic (P1712 found in Q1)
queryString = """
SELECT DISTINCT ?film ?o
{  
?film wdt:P179 wd:Q1364022. #franchise
?film wdt:P1712 ?o.
}
LIMIT 50
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('o', 'movie/halloween-1978')]
[('film', 'http://www.wikidata.org/entity/Q1050815'), ('o', 'movie/halloween-h20-20-years-later')]
[('film', 'http://www.wikidata.org/entity/Q176488'), ('o', 'movie/halloween-resurrection')]
[('film', 'http://www.wikidata.org/entity/Q909063'), ('o', 'movie/halloween')]
[('film', 'http://www.wikidata.org/entity/Q850312'), ('o', 'movie/halloween-ii')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('o', 'movie/halloween-iii-season-of-the-witch')]
[('film', 'http://www.wikidata.org/entity/Q959853'), ('o', 'movie/halloween-4-the-return-of-michael-myers')]
[('film', 'http://www.wikidata.org/entity/Q976728'), ('o', 'movie/halloween-5-the-revenge-of-michael-myers')]
[('film', 'http://www.wikidata.org/entity/Q908690'), ('o', 'movie/halloween-ii-2009')]
[('film', 'http://www.wikidata.org/entity/Q1138055'), ('o', 'movie/halloween-the-curse-of-michael-myers')]


10

Also the different IDs don't look to be useful, i think the best way is to see each film how many time has been nominated/won some awards using
P1411 (found in Q1) and P166 (found in Q11).

In [20]:
#Q40
#also the different IDs don't look to be useful, i think the best way is to see each film how many time has been nominated/won some awards using
#P1411 (found in Q1) and P166 (found in Q11)
queryString = """
SELECT ?film ?nomination ?award WHERE{

{SELECT ?film (COUNT(?o) as ?nomination) WHERE
{  
?film wdt:P179 ?franchise;
      wdt:P1411 ?o.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}
}
GROUP BY ?film}

{SELECT ?film (COUNT(?o) as ?award) WHERE
{  
?film wdt:P179 ?franchise;
      wdt:P166 ?o.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}
}
GROUP BY ?film}
}
ORDER BY ?award ?nomination
"""
print("Results")
run_query(queryString)

Results
Empty


0

Retrieve just nominations for both films.

In [21]:
#Q41
#just nomination
queryString = """
SELECT ?film (COUNT(?o) as ?nomination) WHERE
{  
?film wdt:P179 ?franchise;
      wdt:P1411 ?o.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}
}
GROUP BY ?film
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('nomination', '2')]


1

Retrieve awards won by each film.

In [23]:
#Q42
#just awards won
queryString = """
SELECT ?film (COUNT(?o) as ?awards) WHERE
{  
?film wdt:P179 ?franchise;
      wdt:P166 ?o.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}
}
GROUP BY ?film
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('awards', '1')]


1

Retrieve the awards for what Friday 13th films have been nominated.

In [5]:
#Q43
#get film and for what it has been nominated/what it won
queryString = """
SELECT ?film ?fname ?nomination ?nname ?award ?aname WHERE
{  
?film wdt:P179 ?franchise.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}
?film <http://schema.org/name> ?fname .
{ { 
?film wdt:P1411 ?nomination.
?nomination <http://schema.org/name> ?nname .
} UNION { 
?film wdt:P166 ?award.
?award <http://schema.org/name> ?aname .} }
}
LIMIT 10
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('fname', 'Friday the 13th'), ('nomination', 'http://www.wikidata.org/entity/Q1780634'), ('nname', 'Golden Raspberry Award for Worst Picture')]
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('fname', 'Friday the 13th'), ('nomination', 'http://www.wikidata.org/entity/Q977915'), ('nname', 'Golden Raspberry Award for Worst Supporting Actress')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('fname', 'Halloween'), ('award', 'http://www.wikidata.org/entity/Q823422'), ('aname', 'National Film Registry')]


3

Very interesting this golden raspberry award, just for curiosity i want to take a quick look at this (wd:Q1780634).

In [6]:
#Q44
#very interesting this golden raspberry award, just for curiosity i want to take a quick look at this (wd:Q1780634)
queryString = """
SELECT ?p ?pname ?o ?oname WHERE
{  
wd:Q1780634 ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .
}
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q15455603'), ('oname', 'Buzz Feitshans')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q185724'), ('oname', 'Mike Myers')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q381545'), ('oname', 'Elie Samaha')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q2563751'), ('oname', 'Jon Peters')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q3999144'), ('oname', 'Triumph Films')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://www.wikidata.org/entity/Q7482626'), ('oname', 'Steve Tisch')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('o', 'http://ww

75

In [7]:
#Q45
#very interesting this golden raspberry award, just for curiosity i want to take a quick look at this (wd:Q1780634)
queryString = """
SELECT ?p ?pname ?o ?oname WHERE
{

wd:Q1780634 wdt:P31 ?s.
?s ?p ?o.

#Labels
?p <http://schema.org/name> ?pname .
?o <http://schema.org/name> ?oname .
}
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part'), ('o', 'http://www.wikidata.org/entity/Q1535134'), ('oname', 'Golden Raspberry Award for Worst Visual Effects')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q107655869'), ('oname', 'group of awards')]
[('p', 'http://www.wikidata.org/prop/direct/P8225'), ('pname', 'is metaclass for'), ('o', 'http://www.wikidata.org/entity/Q4220917'), ('oname', 'film award')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q4220917'), ('oname', 'film award')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pname', 'subclass of'), ('o', 'http://www.wikidata.org/entity/Q38033430'), ('oname', 'class of award')]
[('p', 'http://www.

10

I notice that exists also the reverse path (from award to movie) using P1346, let's use it to check if movies won something else.

In [8]:
#Q46
#i notice that exists also the reverse path (from award to movie) using P1346, let's use it to check if movies won something else
queryString = """
SELECT ?film (COUNT(?o) as ?awards) WHERE
{  
?film wdt:P179 ?franchise.
?award wdt:P1346 ?film.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}
}
GROUP BY ?film
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
Empty


0

No results makes sense since all the entities related using P1346 are actors or productors
<h3> TASK 6 ENDS HERE</h3>
I used the number of nominations and awards won to measure how much the crititcs liked the films belonging to the two franchises. 
Friday 13th has been nominated two times for Golden Raspberry Award. Unexpectedly this is not a good thing because it was for worst picture and worst
actress, so I'd say that critics liked Halloween franchise more than Friday 13th one.

<h3> TASK 7 STARTS HERE </h3>
Investigate which franchise costed more 

In Q1 i didn't find a predicate related to production cost, i try to check all predicates for all films.

In [11]:
#Q47
#in Q1 i didn't find a predicate related to production cost, i try to check all predicates for all films.
queryString = """
SELECT DISTINCT ?p ?pname WHERE
{  
?film wdt:P179 ?franchise;
      ?p ?o.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}

#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2438'), ('pname', 'narrator')]
[('p', 'http://www.wikidata.org/prop/direct/P1040'), ('pname', 'film editor')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P135'), ('pname', 'movement')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1434'), ('pname', 'takes place in fictional universe')]
[('p', 'http://www.wikidata.org/prop/direct/P144'), ('pname', 'based on')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('

100

Exclude non-literals

In [12]:
#Q48
#i found cost (P2130) and bugdet (P2769)
#filter some properties excluding the non-literals
queryString = """
SELECT DISTINCT ?p ?pname WHERE
{  
?film wdt:P179 ?franchise;
      ?p ?o.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}
FILTER(isLiteral(?o))

#Labels
?p <http://schema.org/name> ?pname .

}
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('pname', 'Box Office Mojo film ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('pname', 'AlloCiné film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('pname', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P1562'), ('pname', 'AllMovie title ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1712'), ('pname', 'Metacritic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1804'), ('pname', 'DNF film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1874'), ('pname', 'Netflix ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1970'), ('pname', 'MovieMeter film ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2047'), ('pname', 'duration')]
[('p', 'http://www.wikidata.org/prop/direct/P2130'), ('p

83

Check how many movies have cost (P2130) and bugdet (P2769).

In [14]:
#Q49
#check how many movies have cost (P2130) and bugdet (P2769)
queryString = """
SELECT ?film ?fname ?cost ?budget WHERE
{  
?film wdt:P179 ?franchise.
VALUES ?franchise {wd:Q1364022 wd:Q1138878}

{ { ?film wdt:P2130 ?cost.} UNION { ?film wdt:P2769 ?budget.} }

?film <http://schema.org/name> ?fname .

}
ORDER BY DESC(?cost)
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1454799'), ('fname', 'Friday the 13th Part VI: Jason Lives'), ('cost', '3000000')]
[('film', 'http://www.wikidata.org/entity/Q1454798'), ('fname', 'Friday the 13th Part VII: The New Blood'), ('cost', '2800000')]
[('film', 'http://www.wikidata.org/entity/Q1243019'), ('fname', 'Friday the 13th: The Final Chapter'), ('cost', '2600000')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('fname', 'Friday the 13th Part III'), ('cost', '2250000')]
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('fname', 'Friday the 13th Part 2'), ('cost', '1250000')]
[('film', 'http://www.wikidata.org/entity/Q221103'), ('fname', 'Halloween'), ('cost', '325000')]
[('film', 'http://www.wikidata.org/entity/Q176488'), ('fname', 'Halloween: Resurrection'), ('budget', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q909063'), ('fname', 'Halloween'), ('budget', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('fname', 'Halloween 

11

Some films have cost, other budget. i still don't get the difference. Check if some films have both and how are different.

In [15]:
#Q50
#some films have cost, other budget. i still don't get the difference. check if some films have both and how are different
queryString = """
SELECT ?film ?fname ?cost ?budget WHERE
{  

?film wdt:P31 wd:Q11424; #film
      wdt:P2130 ?cost;
      wdt:P2769 ?budget.

?film <http://schema.org/name> ?fname .

}
ORDER BY DESC(?cost)
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q21637453'), ('fname', 'First Time'), ('cost', '400000000'), ('budget', '400000000')]
[('film', 'http://www.wikidata.org/entity/Q21500755'), ('fname', 'Blade Runner 2049'), ('cost', '150000000'), ('budget', '242000000')]
[('film', 'http://www.wikidata.org/entity/Q213081'), ('fname', 'Star Trek'), ('cost', '150000000'), ('budget', '150000000')]
[('film', 'http://www.wikidata.org/entity/Q18547944'), ('fname', 'The Martian'), ('cost', '108000000'), ('budget', '108000000')]
[('film', 'http://www.wikidata.org/entity/Q26465665'), ('fname', 'Life'), ('cost', '58000000'), ('budget', '58000000')]
[('film', 'http://www.wikidata.org/entity/Q30321629'), ('fname', 'All the Money in the World'), ('cost', '50000000'), ('budget', '50000000')]
[('film', 'http://www.wikidata.org/entity/Q164417'), ('fname', 'Corpse Bride'), ('cost', '40000000'), ('budget', '40000000')]
[('film', 'http://www.wikidata.org/entity/Q30011713'), ('fname', 'Game Night'), ('cost'

23

Generally speaking cost and budget have the same value so i will consider them as equivalent predicates.
Compare the five most expensive films from Friday 13th franchise with the top 5 from Hallowen one.

In [17]:
#Q51
#generally speaking cost and budget have the same value so i will consider them as equivalent predicates
#compare the five most expensive films from Friday 13th franchise with the top 5 from Hallowen one
queryString = """
SELECT ?halloween ?friday WHERE
{
{SELECT (SUM(?budget) AS ?halloween) WHERE{
#top 5 from Halloween have P2769
?h wdt:P179 wd:Q1364022; 
      wdt:P2769 ?budget. }}
      
{SELECT (SUM(?cost) AS ?friday) WHERE{
#top 5 from Halloween have P2769
?f wdt:P179 wd:Q1138878; 
      wdt:P2130 ?cost. }}
}
LIMIT 25
"""
print("Results")
run_query(queryString)

Results
[('halloween', '40500000'), ('friday', '11900000')]


1

In [19]:
#Q52
#last check on franchise to see if there are other useful properties (wd:Q1364022)
queryString = """
SELECT DISTINCT ?p ?pname WHERE
{
wd:Q1364022 ?p ?o.
?p <http://schema.org/name> ?pname .
}
LIMIT 100
"""
print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P2530'), ('pname', 'Box Office Mojo franchise ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('pname', 'Rotten Tomatoes ID')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('pname', 'genre')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pname', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1476'), ('pname', 'title')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('pname', 'different from')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P364'), ('pname', 'original language of film or TV show')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('pname', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('pname', 'country of origin')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('pname', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pn

15

<h3> TASK 7 ENDS HERE </h3>
I found out P2130 (cost) and P2769 (budget). Since only 11 movies have one of these two predicates, I computed the total cost of the top 5 most 
expensive movies for each franchise and as a result, I obtained that Halloween franchise required four times the budget required by Friday 13th.